In [ ]:
import gc
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
import torch
from datetime import datetime
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Install qwen-tts
!pip install -q qwen-tts soundfile

In [ ]:
# Configuration parameters (will be updated by deploy script)
TEXT = "Testing Kaggle GPU without instructions."
VOICE_ID = "Ryan"
OUTPUT_FILE = "qwen_gpu.wav"


In [ ]:
print(f"Starting Qwen TTS generation...")
print(f"Text: {TEXT}")
print(f"Voice: {VOICE_ID}")
print(f"Output: {OUTPUT_FILE}")

In [ ]:
# Load Qwen TTS model
from qwen_tts import Qwen3TTSModel

print("Loading Qwen 3 TTS model...")
model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded successfully")

In [ ]:
# Generate audio
print(f"Generating audio for: {TEXT[:50]}...")

audio_output = model.generate(
    text=TEXT,
    speaker=VOICE_ID,
    temperature=1.0,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.0,
    max_new_tokens=2048,
    do_sample=True,
)

print("Audio generation complete")

In [ ]:
# Save audio to file
import soundfile as sf
import numpy as np

# Extract audio data and sample rate
if hasattr(audio_output, 'audio'):
    audio_data = audio_output.audio
    sample_rate = audio_output.sample_rate
else:
    audio_data = audio_output
    sample_rate = 24000  # Default Qwen sample rate

# Convert to numpy if tensor
if isinstance(audio_data, torch.Tensor):
    audio_data = audio_data.cpu().numpy()

# Ensure audio is in correct format
if audio_data.ndim > 1:
    audio_data = audio_data.squeeze()

print(f"Audio shape: {audio_data.shape}, dtype: {audio_data.dtype}")
print(f"Sample rate: {sample_rate}")

# Save to file
sf.write(OUTPUT_FILE, audio_data, sample_rate)
print(f"Audio saved to: {OUTPUT_FILE}")

In [ ]:
# Verify output file exists
output_path = Path(OUTPUT_FILE)
if output_path.exists():
    file_size = output_path.stat().st_size
    print(f"✓ Output file created: {OUTPUT_FILE} ({file_size} bytes)")
else:
    print(f"✗ ERROR: Output file not found: {OUTPUT_FILE}")

In [ ]:
# Cleanup
del model
torch.cuda.empty_cache()
gc.collect()
print("Cleanup complete")